In [ ]:
import pandas
import os

# This query represents dataset "prescription_dates" for domain "drug" and was generated for All of Us Controlled Tier Dataset v7
dataset_28907756_drug_sql = """
    SELECT
        d_exposure.person_id,
        d_standard_concept.concept_name as standard_concept_name,
        d_exposure.drug_exposure_start_datetime,
        d_exposure.drug_exposure_end_datetime 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.drug_exposure` d_exposure 
        WHERE
            (
                drug_concept_id IN  (
                    SELECT
                        DISTINCT ca.descendant_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria_ancestor` ca 
                    JOIN
                        (
                            select
                                distinct c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (
                                    select
                                        cast(cr.id as string) as id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                    WHERE
                                        concept_id IN (
                                            21604686
                                        ) 
                                        AND full_text LIKE '%_rank1]%'
                                ) a 
                                    ON (
                                        c.path LIKE CONCAT('%.',
                                    a.id,
                                    '.%') 
                                    OR c.path LIKE CONCAT('%.',
                                    a.id) 
                                    OR c.path LIKE CONCAT(a.id,
                                    '.%') 
                                    OR c.path = a.id) 
                                WHERE
                                    is_standard = 1 
                                    AND is_selectable = 1
                                ) b 
                                    ON (
                                        ca.ancestor_id = b.concept_id
                                    )
                            )
                        )  
                        AND (
                            d_exposure.PERSON_ID IN (
                                SELECT
                                    distinct person_id  
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                            WHERE
                                cb_search_person.person_id IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                concept_id IN (
                                                    SELECT
                                                        DISTINCT ca.descendant_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria_ancestor` ca 
                                                    JOIN
                                                        (
                                                            select
                                                                distinct c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        concept_id IN (715939) 
                                                                        AND full_text LIKE '%_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    is_standard = 1 
                                                                    AND is_selectable = 1
                                                                ) b 
                                                                    ON (
                                                                        ca.ancestor_id = b.concept_id
                                                                    )
                                                            ) 
                                                            AND is_standard = 1
                                                        )
                                                ) criteria 
                                        GROUP BY
                                            criteria.person_id,
                                            criteria.concept_id 
                                        HAVING
                                            COUNT(criteria.person_id) >= 3 
                                        ) 
                                    ))) d_exposure 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_standard_concept 
                                    ON d_exposure.drug_concept_id = d_standard_concept.concept_id"""

patients_trimmed = pandas.read_gbq(
    dataset_28907756_drug_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

patients_trimmed.head(5)

In [ ]:
import os
import subprocess
import numpy as np
import pandas as pd

In [ ]:
patients_trimmed['standard_concept_name'] = patients_trimmed['standard_concept_name'].str.lower()
patients_trimmed = patients_trimmed.sort_values(by=['standard_concept_name'])
patients_trimmed['standard_concept_name'].unique()

In [ ]:
patients_trimmed = patients_trimmed[~patients_trimmed['standard_concept_name'].str.contains('mg/ml / arginine', regex=False)]
patients_trimmed_escitalopram = patients_trimmed[patients_trimmed['standard_concept_name'].str.contains('escitalopram', regex=False)]
patients_trimmed_escitalopram['standard_concept_name']
patients_trimmed_no_escitalopram = patients_trimmed[~patients_trimmed['standard_concept_name'].str.contains('escitalopram', regex=False)]
print(patients_trimmed_escitalopram['standard_concept_name'].unique())
print(patients_trimmed_no_escitalopram['standard_concept_name'].unique())

In [ ]:
keywords = [
    'clomipramine', 'doxepin', 'nortriptyline', 'desipramine', 'isocarboxazid', 
    'fluvoxamine', 'protriptyline', 'esketamine', 'hydroxytryptophan', 'amitriptyline', 
    'maprotiline', 'escitalopram', 'sertraline', 'fluoxetine', 'venlafaxine', 
    'tranylcypromine', 'paroxetine', 'milnacipran', 'amoxapine', 'phenelzine', 
    'desvenlafaxine', 'bupropion', 'citalopram', 'trazodone', 'duloxetine', 
    'imipramine', 'trimipramine', 'tryptophan', 'vortioxetine', 'mirtazapine', 
    'nefazodone', 'vilazodone'
]
def replace_medication_names(column, keywords):
    """
    Replaces various medication names with their root medication names.

    Args:
    column (pd.Series): The pandas column containing medication names.

    Returns:
    pd.Series: The modified pandas column with root medication names.
    """
    for keyword in keywords:
        column = column.str.replace(rf'.*{keyword}.*', keyword, regex=True)
    return column

In [ ]:
patients_trimmed_no_escitalopram['standard_concept_name'] = replace_medication_names(patients_trimmed_no_escitalopram['standard_concept_name'], keywords)
patients_trimmed_escitalopram['standard_concept_name'] = replace_medication_names(patients_trimmed_escitalopram['standard_concept_name'], ['escitalopram'])


In [ ]:
patients_simple_med = pd.concat([patients_trimmed_escitalopram, patients_trimmed_no_escitalopram])

print(f"{patients_simple_med['standard_concept_name'].unique()}")
print(f"{patients_simple_med.info()}")

In [ ]:
import matplotlib.pyplot as plt

# Assuming patients_trimmed['standard_concept_name'] is already processed as required

# Calculating the distribution (count) of each unique value in the 'standard_concept_name' column
value_counts = patients_simple_med['standard_concept_name'].value_counts()

# Plotting the distribution
plt.figure(figsize=(12, 8))
value_counts.plot(kind='bar')
plt.title('Distribution of Medications in Patients_Trimmed')
plt.xlabel('Medication')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')  # Rotating the x-axis labels for better readability
plt.tight_layout()  # Adjust layout to fit all labels
plt.show()

In [ ]:
def check_escitalopram_as_recent_medication(df):
    """
    Determines if escitalopram is part of the most recent prescriptions for each patient
    and lists the unique medications prescribed on the most recent prescription date. Also,
    checks if escitalopram was the first medication prescribed.

    Args:
    df (pd.DataFrame): DataFrame containing patient medication data.

    Returns:
    pd.DataFrame: DataFrame with columns [person_id, successful, final_meds, first_prescribed], where 
                  'successful' is True if escitalopram was among the most recent medications prescribed,
                  'final_meds' is a list of unique medications prescribed on the most recent date,
                  and 'first_prescribed' is True if escitalopram was the first medication prescribed.
    """
    # Converting 'drug_exposure_start_datetime' to datetime and extracting the date part
    df['drug_exposure_start_date'] = pd.to_datetime(df['drug_exposure_start_datetime']).dt.date

    # Grouping by person_id and finding the latest and earliest prescription dates
    latest_prescription_date = df.groupby('person_id')['drug_exposure_start_date'].max()
    earliest_prescription_date = df.groupby('person_id')['drug_exposure_start_date'].min()

    # Joining the latest and earliest prescription dates with the original DataFrame
    df_latest = df.join(latest_prescription_date, on='person_id', rsuffix='_latest')
    df_earliest = df.join(earliest_prescription_date, on='person_id', rsuffix='_earliest')

    # Filtering to keep only rows where the prescription date matches the latest and earliest prescription dates
    df_latest = df_latest[df_latest['drug_exposure_start_date'] == df_latest['drug_exposure_start_date_latest']]
    df_earliest = df_earliest[df_earliest['drug_exposure_start_date'] == df_earliest['drug_exposure_start_date_earliest']]

    # Getting unique medications prescribed on the latest date for each patient
    final_meds = df_latest.groupby('person_id')['standard_concept_name'].unique()

    # Checking if escitalopram is among the most recent medications
    df_success = df_latest.groupby('person_id')['standard_concept_name'].apply(lambda x: 'escitalopram' in x.values)

    # Checking if escitalopram was the first medication prescribed
    df_first_prescribed = df_earliest.groupby('person_id')['standard_concept_name'].apply(lambda x: 'escitalopram' in x.values)

    # Creating the result DataFrame with 'successful', 'final_meds', and 'first_prescribed' columns
    result_df = pd.DataFrame({
        'person_id': df_success.index,
        'successful': df_success.values,
        'final_meds': final_meds.values,
        'first_prescribed': df_first_prescribed.values
    })

    return result_df



result = check_escitalopram_as_recent_medication(patients_simple_med)
print(result.head())

In [ ]:
import os
import subprocess
import numpy as np
import pandas as pd




In [ ]:
# This snippet assumes that you run setup first

# This code lists objects in your Google Bucket

# Get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# List objects in the bucket
print(subprocess.check_output(f"gsutil ls -r {my_bucket}", shell=True).decode('utf-8'))




In [ ]:
# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = result  

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'patient_success.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr
